In [5]:
%pip install opencv-python
%pip install cvlib
%pip install scikit-learn
%pip install joblib
%pip install pandas
%pip install numpy
# If not installed already (and for Python 3.11):
%pip install tensorflow==2.20.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\sbval\PycharmProjects\MLSN-Team-6\cvenv\Scripts\python.exe -m pip install --upgrade pip


In [1]:
import cv2
import cvlib as cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
import pandas as pd
import numpy as np
import os

In [2]:

path = r"C:\Users\sbval\PycharmProjects\MLSN-Team-6\images_flat"

import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def load_images_from_folder(base_path):
    data = {'image' : [], 'emotion': []}

    for emotion in os.listdir(base_path):
        emotion_folder = os.path.join(base_path, emotion)

        if not os.path.isdir(emotion_folder):
            continue

        for file in os.listdir(emotion_folder):
            if file.lower().endswith(('.png','.jpg','.jpeg')):
                img_path = os.path.join(emotion_folder,file)
                img = cv2.imread(img_path)

                if img is None:
                    continue

                img = cv2.resize(img, (100, 100))
                data['image'].append(img)
                data['emotion'].append(emotion)

    return pd.DataFrame(data)




In [3]:
train_data = load_images_from_folder(path)


from sklearn.model_selection import train_test_split

X = train_data["image"]
y = train_data["emotion"]


X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
X_train_flat = [img.flatten() for img in X_train]
X_test_flat = [img.flatten() for img in X_test]



In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

base_rf = RandomForestClassifier(
    class_weight="balanced",
    random_state=42,
    n_jobs=1   # ✅ only 1 worker — lowest memory footprint
)

# ✅ smaller, sensible search space
param_distributions = {
    "n_estimators": [50, 100, 150, 200, 250, 300, 350],       # ✅ lower tree count
    "max_depth": [10, 20, 30, 40, 50, 100, None],      # ✅ realistic depths
    "max_features": ["sqrt", "log2"],     # ✅ simpler + faster
    "min_samples_split": [2, 4, 6, 8 , 10],
    "min_samples_leaf": [1, 2, 3, 4 ,5]
}

rf_search = RandomizedSearchCV(
    estimator=base_rf,
    param_distributions=param_distributions,
    n_iter=15,        # ✅ drastically fewer fits
    cv=2,             # ✅ half the memory + time
    scoring="accuracy",
    verbose=2,
    n_jobs=1,         # ✅ prevents RAM duplication across workers
    random_state=42
)

rf_search.fit(X_train_flat, Y_train)

print("Best hyperparameters found:")
print(rf_search.best_params_)
print(f"Best cross-validated score: {rf_search.best_score_:.4f}")


Fitting 2 folds for each of 15 candidates, totalling 30 fits


KeyboardInterrupt: 

In [5]:
model = RandomForestClassifier(
  n_estimators=250,      # 100–200 is a good sweet spot
    max_depth=35,         # limit depth to avoid huge trees
    max_features="sqrt",  # classic choice for classification
    min_samples_split=4,  # small regularization
    min_samples_leaf=2,   # prevents tiny leaves / overfitting
    bootstrap=True,
    class_weight="balanced",  # important if classes aren't equal
    n_jobs=2,             # or 2 if your RAM is okay
    random_state=42
)

model.fit(X_train_flat, Y_train)



,n_estimators,250
,criterion,'gini'
,max_depth,35
,min_samples_split,4
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test_flat)
acc = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {acc:.4f}")


from sklearn.metrics import classification_report

print(classification_report(Y_test,Y_pred))

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test,Y_pred)
print(cm)


Accuracy: 0.5084
              precision    recall  f1-score   support

     Neutral       0.50      0.39      0.44       132
       happy       0.55      0.47      0.51       135
         sad       0.41      0.52      0.46       146
    surprise       0.63      0.66      0.64       120

    accuracy                           0.51       533
   macro avg       0.52      0.51      0.51       533
weighted avg       0.52      0.51      0.51       533

[[52 17 50 13]
 [15 64 33 23]
 [33 27 76 10]
 [ 5  9 27 79]]


In [53]:
model_filename = 'emotion_detection_model.pkl'
joblib.dump(model, model_filename)





['emotion_detection_model.pkl']